In [1]:
import json

def find_json_snippet(raw_snippet):
	"""
	find_json_snippet tries to find JSON snippets in a given raw_snippet string
	"""
	json_parsed_string = None

	json_start_index = raw_snippet.find('[')
	json_end_index = raw_snippet.rfind(']')

	if json_start_index >= 0 and json_end_index >= 0:
		json_snippet = raw_snippet[json_start_index:json_end_index+1]
		try:
			json_parsed_string = json.loads(json_snippet, strict=False)
		except:
			raise ValueError('......failed to parse string into JSON format')
	else:
		raise ValueError('......No JSON code snippet found in string.')

	return json_parsed_string

In [3]:
from typing import List, Dict

def format_response(responses: List[Dict[str, str]]):
    final_instruction_answer_pair = []

    for response in responses:
        user_response_dict = {}
        assistant_response_dict = {}
        user_response_dict["content"] = response["instruction"]
        user_response_dict["role"] = "user"
        assistant_response_dict["content"] = response["response"]
        assistant_response_dict["role"] = "assistant"

        final_instruction_answer_pair.append([user_response_dict, assistant_response_dict])

    return final_instruction_answer_pair

In [4]:
import glob 

all_jsons = glob.glob("*.json")
all_jsons

['gemini_results_0.json',
 'gemini_results_1.json',
 'gemini_results_4.json',
 'gemini_results_2.json',
 'gemini_results_3.json']

In [5]:
with open(all_jsons[0]) as f:
    response_sample = json.load(f)
response_sample

[{'candidates': [{'content': {'parts': [{'text': '[\n    {\n        "instruction": "Create a function called \'get_total\' that takes a list of numbers as a parameter and returns the sum of all the numbers.",\n        "response": "```python\\ndef get_total(numbers: list) -> int:\\n    \\"\\"\\"Calculates the sum of a list of numbers.\\"\\"\\"\\n    total = 0\\n    for number in numbers:\\n        total += number\\n    return total```"\n    },\n    {\n        "instruction": "Write a Python program that takes a list of integers and prints the largest and smallest numbers in the list.",\n        "response": "```python\\nnums = [3, 6, 9, 12, 15, 18]\\n\\n# Find the largest number\\nlargest = max(nums)\\n\\n# Find the smallest number\\nsmallest = min(nums)\\n\\n# Print the results\\nprint(\'Largest number:\', largest)\\nprint(\'Smallest number:\', smallest)```"\n    },\n    {\n        "instruction": "Create a Python dictionary that represents a student\'s information, including their name, 

In [6]:
len(response_sample)

5

In [8]:
response_sample[0]["candidates"][0]["content"]["parts"][0]["text"]

'[\n    {\n        "instruction": "Create a function called \'get_total\' that takes a list of numbers as a parameter and returns the sum of all the numbers.",\n        "response": "```python\\ndef get_total(numbers: list) -> int:\\n    \\"\\"\\"Calculates the sum of a list of numbers.\\"\\"\\"\\n    total = 0\\n    for number in numbers:\\n        total += number\\n    return total```"\n    },\n    {\n        "instruction": "Write a Python program that takes a list of integers and prints the largest and smallest numbers in the list.",\n        "response": "```python\\nnums = [3, 6, 9, 12, 15, 18]\\n\\n# Find the largest number\\nlargest = max(nums)\\n\\n# Find the smallest number\\nsmallest = min(nums)\\n\\n# Print the results\\nprint(\'Largest number:\', largest)\\nprint(\'Smallest number:\', smallest)```"\n    },\n    {\n        "instruction": "Create a Python dictionary that represents a student\'s information, including their name, age, and courses.",\n        "response": "```pyth

In [10]:
eval(response_sample[0]["candidates"][0]["content"]["parts"][0]["text"])

[{'instruction': "Create a function called 'get_total' that takes a list of numbers as a parameter and returns the sum of all the numbers.",
  'response': '```python\ndef get_total(numbers: list) -> int:\n    """Calculates the sum of a list of numbers."""\n    total = 0\n    for number in numbers:\n        total += number\n    return total```'},
 {'instruction': 'Write a Python program that takes a list of integers and prints the largest and smallest numbers in the list.',
  'response': "```python\nnums = [3, 6, 9, 12, 15, 18]\n\n# Find the largest number\nlargest = max(nums)\n\n# Find the smallest number\nsmallest = min(nums)\n\n# Print the results\nprint('Largest number:', largest)\nprint('Smallest number:', smallest)```"},
 {'instruction': "Create a Python dictionary that represents a student's information, including their name, age, and courses.",
  'response': "```python\nstudent = {\n    'name': 'Alice Smith',\n    'age': 20,\n    'courses': ['Python', 'Data Structures', 'Machine

In [12]:
all_formatted_responses = format_response(
    eval(response_sample[0]["candidates"][0]["content"]["parts"][0]["text"])
)
all_formatted_responses[0]

[{'content': "Create a function called 'get_total' that takes a list of numbers as a parameter and returns the sum of all the numbers.",
  'role': 'user'},
 {'content': '```python\ndef get_total(numbers: list) -> int:\n    """Calculates the sum of a list of numbers."""\n    total = 0\n    for number in numbers:\n        total += number\n    return total```',
  'role': 'assistant'}]

In [14]:
from datasets import Dataset

prompts = ["gemini-generated"] * len(all_formatted_responses)
prompt_ids = ["gemini-generated"] * len(all_formatted_responses)
categories = ["Coding"] * len(all_formatted_responses)

dataset = Dataset.from_dict({
    "prompt": prompts,
    "prompt_id": prompt_ids,
    "messages": all_formatted_responses,
    "category": categories
})
dataset

Dataset({
    features: ['prompt', 'prompt_id', 'messages', 'category'],
    num_rows: 5
})

In [15]:
dataset[0]

{'prompt': 'gemini-generated',
 'prompt_id': 'gemini-generated',
 'messages': [{'content': "Create a function called 'get_total' that takes a list of numbers as a parameter and returns the sum of all the numbers.",
   'role': 'user'},
  {'content': '```python\ndef get_total(numbers: list) -> int:\n    """Calculates the sum of a list of numbers."""\n    total = 0\n    for number in numbers:\n        total += number\n    return total```',
   'role': 'assistant'}],
 'category': 'Coding'}